# Data preparation

In [31]:
import pandas as pd
import numpy as np

DF = pd.read_csv('../data/export_with_basket_2017.csv')
DF.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3,80391417
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2,81849313
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2,81849313
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895


Let us now create a feature that will be our target. Perhaps this step is not even required at this stage since we will be using our data for re-ranking of the suggested apps.

So, what will be the right thing to do:
1. Super simple approach is to select the latest item and predict it
    a. So we take the latest transaction in time, select form there items that were bought and use that as a target label. __ in this case we have the situation that there is no one to one mapping.

In [32]:
DF.columns

Index(['SCM_N_ORDER_POS_NR', 'SCM_N_ORDER_NR', 'SCM_N_ORDER_NR_NC',
       'TDT_T_KEY_ORDER_DATE', 'ART_N_KEY_ARTICLE', 'PPR_N_KEY_PHYS_PRODUCT',
       'CUS_N_KEY_CUSTOMER', 'SCM_D_QUANTITY', 'PPR_N_PHY_ARTICLE_ID',
       'PPR_N_PHY_PRODUCT_ID', 'ART_N_KEY_ARTICLE_1', 'ART_C_ART_NUMBER',
       'ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION', 'ART_V_ART_DESCRIPTION_1',
       'ART_V_PG_LEVEL3_DESC', 'ART_V_PG_LEVEL4_DESC', 'ART_V_PG_LEVEL5_DESC',
       'ART_V_PG_LEVEL6_DESC', 'ART_V_MANUFACTURER_DESC', 'PPR_ART_CLASS_DESC',
       'PPR_N_LEVEL2_ID', 'SCM_N_ORDER_NR_1'],
      dtype='object')

Let us write a function that will map to a separate "target" column the items that were bought during the last order that was available

In addition we will split the order into each item into new line

In [33]:
display(DF.groupby('CUS_N_KEY_CUSTOMER').apply(lambda x: (x['TDT_T_KEY_ORDER_DATE'].max(),x['TDT_T_KEY_ORDER_DATE'].min())))

CUS_N_KEY_CUSTOMER
22          (28-FEB-17, 05-JUN-17)
44          (22-FEB-17, 02-JAN-17)
54          (23-SEP-17, 23-SEP-17)
64          (31-JUL-17, 01-APR-17)
84          (26-NOV-17, 06-DEC-17)
125         (28-JUL-17, 02-OCT-17)
132         (08-NOV-17, 08-NOV-17)
176         (21-MAR-17, 03-MAY-17)
205         (30-OCT-17, 02-FEB-17)
208         (28-MAY-17, 01-MAR-17)
221         (23-MAR-17, 06-DEC-17)
237         (20-APR-17, 06-OCT-17)
289         (29-NOV-17, 02-JUL-17)
340         (14-AUG-17, 04-DEC-17)
354         (25-SEP-17, 01-NOV-17)
363         (14-JUN-17, 14-JUN-17)
413         (27-JUN-17, 04-OCT-17)
450         (20-JUL-17, 02-MAY-17)
458         (27-APR-17, 17-SEP-17)
464         (20-MAR-17, 05-AUG-17)
484         (18-JUN-17, 07-SEP-17)
485         (27-NOV-17, 07-APR-17)
502         (27-AUG-17, 01-SEP-17)
508         (30-JUL-17, 03-DEC-17)
512         (19-MAY-17, 01-MAR-17)
514         (13-NOV-17, 03-JUL-17)
522         (31-JAN-17, 01-DEC-17)
556         (05-FEB-17, 05-FEB-17)
5

In [23]:
DF[DF['CUS_N_KEY_CUSTOMER'] == 18887214]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
1084,430580536,100906736,100906736,18-DEC-17,1310734,62776881,18887214,1.0,61213,4903,...,4903.19 Royal Canin Fit 32 - 10 kg,4903.19 Royal Canin Fit 32 - 10 kg,Katzenfutter trocken,Royal Canin,Health Outdoor,Health Outdoor,Royal Canin,Royal Canin,3,100906736


In [ ]:
DF.groupby(['CUS_N_KEY_CUSTOMER', 'SCM_N_ORDER_NR']).agg(['count'])